In [1]:
using GATlab

In [2]:
# Generic test case type that other test cases inherit from

struct TestCase{T, O}
    input::T
    expectedOutput::O
end

In [3]:
# Test case type for task of finding the greatest integer out of three integers.

const FirstTestCase = TestCase{Tuple{Int, Int, Int}, Int}

example1 = FirstTestCase((1,2,3), 3)

struct FirstTestSuiteC <: Model{Tuple{FirstTestCase, Tuple{FirstTestCase, FirstTestCase}}} end

# Function for determining whether two tuples are permutations of each other.

function is_permutation(tuple_l::Tuple, tuple_r::Tuple) :: Bool 
    return sort(collect(tuple_l)) == sort(collect(tuple_r))
end

# Test suite model for the first task
# An object is valid, if it is a valid test case of the task, i.e. if the expected output matches the maximum integer in input tuple.
# A morphism is a tuple of domain and codomain test cases. It is valid, if the domains and codomain test cases are permutations of each other.
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{FirstTestCase, Tuple{FirstTestCase, FirstTestCase}} [model::FirstTestSuiteC] begin

    Ob(x::FirstTestCase) = maximum(x.input) == x.expectedOutput ? x : @fail "Invalid test case."

    function Hom(f::Tuple{FirstTestCase, FirstTestCase}, dom::FirstTestCase, codom::FirstTestCase)
        if is_permutation(dom.input, codom.input)
            return f
        else
            @fail "Test case input arrays are not permutations of each other."
        end
    end

    id(testcase::FirstTestCase) = Tuple{FirstTestCase, FirstTestCase}((testcase, testcase))

    compose(f::Tuple{FirstTestCase, FirstTestCase}, g::Tuple{FirstTestCase, FirstTestCase}) = 
                            f[2] == g[1] ? Tuple{FirstTestCase, FirstTestCase}((f[1], g[2])) : @fail "Domain and codomain of morphisms must match in order to perform composition"
    
    dom(f::Tuple{FirstTestCase, FirstTestCase}) = f[1]
    codom(f::Tuple{FirstTestCase, FirstTestCase}) = f[2]

end

##docsink#304 (generic function with 0 methods)

In [4]:
# Example test cases in the test suite of the first task

a = GATlab.ThThinCategory.Ob[FirstTestSuiteC()](FirstTestCase((1,2,3),3))
b = GATlab.ThThinCategory.Ob[FirstTestSuiteC()](FirstTestCase((1,3,2),3))
c = GATlab.ThThinCategory.Ob[FirstTestSuiteC()](FirstTestCase((3,1,2),3))

f = GATlab.ThThinCategory.Hom[FirstTestSuiteC()]((a,b)::Tuple{FirstTestCase, FirstTestCase}, a, b)
g = GATlab.ThThinCategory.Hom[FirstTestSuiteC()]((b,c)::Tuple{FirstTestCase, FirstTestCase}, b, c)

h = GATlab.ThThinCategory.compose[FirstTestSuiteC()](f,g)

(FirstTestCase((1, 2, 3), 3), FirstTestCase((3, 1, 2), 3))

In [115]:
# Test case for the second task of determining whether an integer is even.

const SecondTestCase = TestCase{Int, Int}

example2 = SecondTestCase(1,0)

struct SecondTestSuiteC <: Model{Tuple{SecondTestCase, Tuple{Int, Int}}} end

# Test suite model for the second task.
# An object is valid, if it is a valid test case of the task, i.e. if the expected output is 1 when input is an even integer and 0 when input is an odd integer.
# A morphism is an addition of an even number to input of the domain test case.
# Composition of morphisms adds the sum of two even numbers from two morphisms. It is valid, if the input of the second test case is equal to applied addition to input of first morphism's test case.
@instance GATlab.ThThinCategory{SecondTestCase, Tuple{Int, Int}} [model::SecondTestSuiteC] begin
    
    Ob(x::SecondTestCase) = abs(x.input % 2 - 1) == x.expectedOutput ? x : @fail "Invalid test case."

    Hom(f::Tuple{Int, Int}, dom::SecondTestCase, codom::SecondTestCase) = f[2] % 2 == 0 && dom.input + f[2] == codom.input && f[1] == dom.input ? f : @fail "Invalid morphism"

    id(testcase::SecondTestCase) = (testcase.input, 0)

    compose(f::Tuple{Int,Int}, g::Tuple{Int,Int}) = f[1] + f[2] == g[1] ? (f[1], g[1] + g[2] - f[1])::Tuple{Int,Int} : @fail "Invalid composition"

    dom(f::Tuple{Int, Int}) = f[1]

    codom(f::Tuple{Int, Int}) = f[1] + f[2]

end

##docsink#460 (generic function with 0 methods)

In [6]:
# Example test cases in the test suite of the second task

a = GATlab.ThThinCategory.Ob[SecondTestSuiteC()](SecondTestCase(1,0))
b = GATlab.ThThinCategory.Ob[SecondTestSuiteC()](SecondTestCase(2,1))
c = GATlab.ThThinCategory.Ob[SecondTestSuiteC()](SecondTestCase(3,0))
d = GATlab.ThThinCategory.Ob[SecondTestSuiteC()](SecondTestCase(5,0))


f = GATlab.ThThinCategory.Hom[SecondTestSuiteC()]((a.input,2)::Tuple{Int, Int}, a, c)
g = GATlab.ThThinCategory.Hom[SecondTestSuiteC()]((c.input,2)::Tuple{Int, Int}, c, d)

h = GATlab.ThThinCategory.compose[SecondTestSuiteC()](f,g)

(1, 4)

In [7]:
# Test case for the third task of calculating the sum of integers in a list.

const ThirdTestCase = TestCase{Vector{Int}, Int}

example3 = ThirdTestCase([1,2,3,4], 10)

struct ThirdTestSuiteC <: Model{Tuple{ThirdTestCase, Tuple{ThirdTestCase, ThirdTestCase}}} end

# Test suite model for third task
# An object is valid, if it is a valid test case of the task, i.e. expected output is equal to the sum of the integers in the input list.
# A morphism is a tuple of domain and codomain test cases. It is valid if the input of codomain test case contains all elements of input in the domain test case.
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{ThirdTestCase, Tuple{ThirdTestCase, ThirdTestCase}} [model::ThirdTestSuiteC] begin
    Ob(x::ThirdTestCase) = sum(x.input) == x.expectedOutput ? x : @fail "Invalid object"

    function Hom(f::Tuple{ThirdTestCase, ThirdTestCase}, dom::ThirdTestCase, codom::ThirdTestCase) 
        A = f[2].input
        B = f[1].input
        
        count_A = Dict()

        for element in A
            count_A[element] = get(count_A, element, 0) + 1
        end

        for element in B
            if !haskey(count_A, element) || count_A[element] == 0
                @fail "Invalid morphism"
            else
                count_A[element] -= 1
            end
        end
    
        return f
    end

    id(x::ThirdTestCase) = (x, x)

    compose(f::Tuple{ThirdTestCase, ThirdTestCase}, g::Tuple{ThirdTestCase, ThirdTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{ThirdTestCase, ThirdTestCase}) = f[1]

    codom(f::Tuple{ThirdTestCase, ThirdTestCase}) = f[2]
end

##docsink#330 (generic function with 0 methods)

In [8]:
# Example test cases in the test suite of the third task

a = GATlab.ThThinCategory.Ob[ThirdTestSuiteC()](ThirdTestCase([1],1))
b = GATlab.ThThinCategory.Ob[ThirdTestSuiteC()](ThirdTestCase([1,3],4))
c = GATlab.ThThinCategory.Ob[ThirdTestSuiteC()](ThirdTestCase([1,3,6],10))

f = GATlab.ThThinCategory.Hom[ThirdTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[ThirdTestSuiteC()]((b,c),b,c) 
id = GATlab.ThThinCategory.id[ThirdTestSuiteC()](b)

composition = GATlab.ThThinCategory.compose[ThirdTestSuiteC()](f,id)

dom = GATlab.ThThinCategory.dom[ThirdTestSuiteC()](composition)
codom = GATlab.ThThinCategory.codom[ThirdTestSuiteC()](composition)

show(dom)
show(codom)

ThirdTestCase([1], 1)ThirdTestCase([1, 3], 4)

In [9]:
# Test case for the fourth task of reversing a finite list.

const FourthTestCase = TestCase{Vector{Int}, Vector{Int}}

example4 = FourthTestCase([1,3,2],[2,3,1])

struct FourthTestSuiteC <: Model{Tuple{FourthTestCase, Tuple{FourthTestCase, FourthTestCase}}} end

# Test suite model for fourth task
# An object is valid, if it is a valid test case of the task, i.e. expected output is the reversed input list
# A morphism is a tuple of domain and codomain test cases. It is valid the codomain test case input contains all elements as a subsequence of the domain test case input.
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{FourthTestCase, Tuple{FourthTestCase, FourthTestCase}} [model::FourthTestSuiteC] begin
    Ob(x::FourthTestCase) = x.input == reverse(x.expectedOutput) ? x : @fail "Invalid object"

    function Hom(f::Tuple{FourthTestCase, FourthTestCase}, dom::FourthTestCase, codom::FourthTestCase)
        dom_input = f[1].input
        codom_input = f[2].input

        dom_expectedoutput = f[1].expectedOutput
        codom_expectedoutput = f[2].expectedOutput

        if length(dom_input) > length(codom_input) || any(dom_input[i] != codom_input[i] for i in 1:length(dom_input))
            @fail "Domain input array is not a subsequence of codomain input array"
        end
    
        if length(dom_expectedoutput) > length(codom_expectedoutput)
           @fail "domain expected output array is not a subsequence of codomain expected output array" 
        end
        
        for i in 1:length(dom_expectedoutput) 
            if dom_expectedoutput[i] != codom_expectedoutput[i+length(codom_expectedoutput)-length(dom_expectedoutput)]
                @fail "domain expected output array is not a subarray of codomain expected output array"
            end
        end

        return f
    end

    id(x::FourthTestCase) = (x, x)

    compose(f::Tuple{FourthTestCase, FourthTestCase}, g::Tuple{FourthTestCase, FourthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{FourthTestCase, FourthTestCase}) = f[1]

    codom(f::Tuple{FourthTestCase, FourthTestCase}) = f[2]
end

##docsink#343 (generic function with 0 methods)

In [10]:
# Example test cases in the test suite of the fourth task

a = GATlab.ThThinCategory.Ob[FourthTestSuiteC()](FourthTestCase([1,2,3],[3,2,1]))
b = GATlab.ThThinCategory.Ob[FourthTestSuiteC()](FourthTestCase([1,3,2],[2,3,1]))
c = GATlab.ThThinCategory.Ob[FourthTestSuiteC()](FourthTestCase([1,3,2,4],[4,2,3,1]))
d = GATlab.ThThinCategory.Ob[FourthTestSuiteC()](FourthTestCase([1,3,2,4,8],[8,4,2,3,1]))

f = GATlab.ThThinCategory.Hom[FourthTestSuiteC()]((b,c), b, c)
g = GATlab.ThThinCategory.Hom[FourthTestSuiteC()]((c,d), c, d)

h = GATlab.ThThinCategory.Hom[FourthTestSuiteC()]((b,d), b, d)
id = GATlab.ThThinCategory.id[FourthTestSuiteC()](a)

compose = GATlab.ThThinCategory.compose[FourthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[FourthTestSuiteC()](compose)
codom = GATlab.ThThinCategory.codom[FourthTestSuiteC()](compose)

show(dom)
show(codom)



FourthTestCase([1, 3, 2], [2, 3, 1])FourthTestCase([1, 3, 2, 4, 8], [8, 4, 2, 3, 1])

In [11]:
# Test case for the fifth task of sorting a finite list in ascending order.

const FifthTestCase = TestCase{Vector{Int}, Vector{Int}}

example5 = FifthTestCase([1,3,2],[1,2,3])

struct FifthTestSuiteC <: Model{Tuple{FifthTestCase, Tuple{FifthTestCase, FifthTestCase}}} end

# Test suite model for fifth task
# An object is valid, if it is a valid test case of the task, i.e. expected output list is equal to sorted input list.
# A morphism is a tuple of domain and codomain test cases. It is valid if input of codomain test case contains all elements of input of domain test case.
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{FifthTestCase, Tuple{FifthTestCase, FifthTestCase}} [model::FifthTestSuiteC] begin
    Ob(x::FifthTestCase) = sort(x.input) == x.expectedOutput ? x : @fail "Invalid object"
    
    function Hom(f::Tuple{FifthTestCase, FifthTestCase}, dom::FifthTestCase, codom::FifthTestCase)
        A = f[2].input
        B = f[1].input
        
        count_A = Dict()

        for element in A
            count_A[element] = get(count_A, element, 0) + 1
        end

        for element in B
            if !haskey(count_A, element) || count_A[element] == 0
                @fail "Invalid morphism"
            else
                count_A[element] -= 1
            end
        end
    
        return f
    end

    id(x::FifthTestCase) = (x, x)

    compose(f::Tuple{FifthTestCase, FifthTestCase}, g::Tuple{FifthTestCase, FifthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{FifthTestCase, FifthTestCase}) = f[1]

    codom(f::Tuple{FifthTestCase, FifthTestCase}) = f[2]
end

##docsink#356 (generic function with 0 methods)

In [12]:
# Example test cases in the test suite of the fifth task

a = GATlab.ThThinCategory.Ob[FifthTestSuiteC()](FifthTestCase([1,3,2],[1,2,3]))
b = GATlab.ThThinCategory.Ob[FifthTestSuiteC()](FifthTestCase([3,1,2],[1,2,3]))
c = GATlab.ThThinCategory.Ob[FifthTestSuiteC()](FifthTestCase([1,3,2,4],[1,2,3,4]))
d = GATlab.ThThinCategory.Ob[FifthTestSuiteC()](FifthTestCase([1,2,2,4],[1,2,2,4]))

f = GATlab.ThThinCategory.Hom[FifthTestSuiteC()]((a,b), a, b)
g = GATlab.ThThinCategory.Hom[FifthTestSuiteC()]((b,c), b, c)

id = GATlab.ThThinCategory.id[FifthTestSuiteC()](a)

composition = GATlab.ThThinCategory.compose[FifthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[FifthTestSuiteC()](composition)

codom = GATlab.ThThinCategory.dom[FifthTestSuiteC()](composition)

show(dom)
show(codom)

TestCase{Vector{

Int64}, Vector{Int64}}([1, 3, 2], [1, 2, 3])TestCase{Vector{Int64}, Vector{Int64}}([1, 3, 2], [1, 2, 3])

In [13]:
# Test case for the sixth task of concatenating finite lists.

const SixthTestCase = TestCase{Vector{Vector{Int}}, Vector{Int}}

example6 = SixthTestCase([[1,2],[3,4]],[1,2,3,4])

struct SixthTestSuiteC <: Model{Tuple{SixthTestCase, Tuple{SixthTestCase, SixthTestCase}}} end

# Test suite model for sixth task
# An object is valid, if it is a valid test case of the task, i.e. expected output is a concatenated list of input lists.
# A morphism is a tuple of domain and codomain test cases. It is valid if concatenated input lists of domain test case is a subsequence of domain test case concatenated input list. 
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{SixthTestCase, Tuple{SixthTestCase, SixthTestCase}} [model::SixthTestSuiteC] begin
    Ob(x::SixthTestCase) = collect(Base.Iterators.Flatten(x.input)) == x.expectedOutput ? x : @fail "Invalid object"

    function Hom(f::Tuple{SixthTestCase, SixthTestCase}, dom::SixthTestCase, codom::SixthTestCase) 
        dom_flatinput = collect(Base.Iterators.Flatten(f[1].input))
        codom_flatinput = collect(Base.Iterators.Flatten(f[2].input))

        if length(dom_flatinput) > length(codom_flatinput) || any(dom_flatinput[i] != codom_flatinput[i] for i in 1:length(dom_flatinput))
            @fail "Domain input array is not a subsequence of codomain input array"
        end

        return f
    end
    
    id(x::SixthTestCase) = (x, x)

    compose(f::Tuple{SixthTestCase, SixthTestCase}, g::Tuple{SixthTestCase, SixthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{SixthTestCase, SixthTestCase}) = f[1]

    codom(f::Tuple{SixthTestCase, SixthTestCase}) = f[2]
end

##docsink#369 (generic function with 0 methods)

In [14]:
# Example test cases in the test suite of the sixth task

a = GATlab.ThThinCategory.Ob[SixthTestSuiteC()](SixthTestCase([[1,3,2]],[1,3,2]))
b = GATlab.ThThinCategory.Ob[SixthTestSuiteC()](SixthTestCase([[1,3],[2]],[1,3,2]))
c = GATlab.ThThinCategory.Ob[SixthTestSuiteC()](SixthTestCase([[1,3],[2,2]],[1,3,2,2]))

f = GATlab.ThThinCategory.Hom[SixthTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[SixthTestSuiteC()]((b,c),b,c)

id = GATlab.ThThinCategory.id[SixthTestSuiteC()](a)

composition = GATlab.ThThinCategory.compose[SixthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[SixthTestSuiteC()](composition)

codom = GATlab.ThThinCategory.codom[SixthTestSuiteC()](composition)

show(dom)
show(codom)


SixthTestCase([[1, 3, 2]], [1, 3, 2])SixthTestCase([[1, 3], [2, 2]], [1, 3, 2, 2])

In [15]:
# Test case for the seventh task of counting the number of occurances of an element in a finite list.

const SeventhTestCase = TestCase{Tuple{Vector{Int}, Int}, Int}

example7 = SeventhTestCase(([1,2,2,3],2),2)

struct SeventhTestSuiteC <: Model{Tuple{SeventhTestCase, Tuple{SeventhTestCase, SeventhTestCase}}} end

# Test suite model for seventh task
# An object is valid, if it is a valid test case of the task, i.e. expceted output matches the count of provided integer in input list.
# A morphism is a tuple of domain and codomain test cases. It is valid if the counted numbers match of domain and codomain test cases and if codomain input array contains all elements of domain input array.
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{SeventhTestCase, Tuple{SeventhTestCase, SeventhTestCase}} [model::SeventhTestSuiteC] begin
    Ob(x::SeventhTestCase) = count(y -> y == x.input[2], x.input[1]) == x.expectedOutput ? x : @fail "Invalid object"

    function Hom(f::Tuple{SeventhTestCase, SeventhTestCase}, dom::SeventhTestCase, codom::SeventhTestCase)
        dom_inputvec = f[1].input[1]
        codom_inputvec = f[2].input[1]

        dom_countednumber = f[1].input[2]
        codom_countednumber = f[1].input[2]
        
        count_codom_inputvec = Dict()

        if dom_countednumber != codom_countednumber
            @fail "Counted number don't match"
        end

        for element in codom_inputvec
            count_codom_inputvec[element] = get(count_codom_inputvec, element, 0) + 1
        end

        for element in dom_inputvec
            if !haskey(count_codom_inputvec, element) || count_codom_inputvec[element] == 0
                @fail "Invalid morphism"
            else
                count_codom_inputvec[element] -= 1
            end
        end
    
        return f
    end

    compose(f::Tuple{SeventhTestCase, SeventhTestCase}, g::Tuple{SeventhTestCase, SeventhTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    id(x::SeventhTestCase) = (x,x)

    dom(f::Tuple{SeventhTestCase, SeventhTestCase}) = f[1]

    codom(f::Tuple{SeventhTestCase, SeventhTestCase}) = f[2]
end

##docsink#382 (generic function with 0 methods)

In [16]:
# Example test cases in the test suite of the seventh task

a = GATlab.ThThinCategory.Ob[SeventhTestSuiteC()](SeventhTestCase(([1,3,2], 2),1))
b = GATlab.ThThinCategory.Ob[SeventhTestSuiteC()](SeventhTestCase(([1,2,3], 2),1))
c = GATlab.ThThinCategory.Ob[SeventhTestSuiteC()](SeventhTestCase(([1,2,3,2], 2),2))

f = GATlab.ThThinCategory.Hom[SeventhTestSuiteC()]((a,b), a, b)
g = GATlab.ThThinCategory.Hom[SeventhTestSuiteC()]((b,c), b, c)

id = GATlab.ThThinCategory.id[SeventhTestSuiteC()](a)

composition = GATlab.ThThinCategory.compose[SeventhTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[SeventhTestSuiteC()](composition)
codom = GATlab.ThThinCategory.codom[SeventhTestSuiteC()](composition)

show(dom)
show(codom)



SeventhTestCase

(([1, 3, 2], 2), 1)SeventhTestCase(([1, 2, 3, 2], 2), 2)

In [17]:
# Test case for the eighth task of finding the most frequently repeating element in a finite list.

const EighthTestCase = TestCase{Vector{Int}, Int} 

example8 = EighthTestCase(([1,2,2,3,4,4]),2)

struct EighthTestSuiteC <: Model{Tuple{EighthTestCase, Tuple{EighthTestCase, EighthTestCase}}} end

# Test suite model for eighth task
# An object is valid, if it is a valid test case of the task, i.e. expected output is one of the highest integers of the input array.
# A morphism is a tuple of domain and codomain test cases. It is valid if the list of codomain test case input is contained by the list of codomain test case input. 
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{EighthTestCase, Tuple{EighthTestCase, EighthTestCase}} [model::EighthTestSuiteC] begin
    function Ob(x::EighthTestCase)
        counts = Dict()
        input_vector = x.input
        expected_output = x.expectedOutput

        if isempty(input_vector)
            return x
        end

        for element in input_vector
            counts[element] = get(counts, element, 0) + 1
        end
        
        max_frequency = maximum(values(counts))

        most_frequent_elements = [element for (element, count) in counts if count == max_frequency]

        if in(expected_output, most_frequent_elements)
            return x
        end

        @fail "Invalid object."
    end

    function Hom(f::Tuple{EighthTestCase, EighthTestCase}, dom::EighthTestCase, codom::EighthTestCase)
        A = f[2].input
        B = f[1].input
        
        count_A = Dict()

        for element in A
            count_A[element] = get(count_A, element, 0) + 1
        end

        for element in B
            if !haskey(count_A, element) || count_A[element] == 0
                @fail "Invalid morphism"
            else
                count_A[element] -= 1
            end
        end
    
        return f
    end

    id(x::EighthTestCase) = (x, x)

    compose(f::Tuple{EighthTestCase, EighthTestCase}, g::Tuple{EighthTestCase, EighthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{EighthTestCase, EighthTestCase}) = f[1]

    codom(f::Tuple{EighthTestCase, EighthTestCase}) = f[2]
end

##docsink#395 (generic function with 0 methods)

In [18]:
# Example test cases in the test suite of the eighth task

a = GATlab.ThThinCategory.Ob[EighthTestSuiteC()](EighthTestCase([1],1))
b = GATlab.ThThinCategory.Ob[EighthTestSuiteC()](EighthTestCase([1,1],1))
c = GATlab.ThThinCategory.Ob[EighthTestSuiteC()](EighthTestCase([1,1,2],1))

f = GATlab.ThThinCategory.Hom[EighthTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[EighthTestSuiteC()]((b,c),b,c)

composition = GATlab.ThThinCategory.compose[EighthTestSuiteC()](f,g)

id = GATlab.ThThinCategory.id[EighthTestSuiteC()](a)

dom = GATlab.ThThinCategory.dom[EighthTestSuiteC()](composition)
codom = GATlab.ThThinCategory.codom[EighthTestSuiteC()](composition)

show(dom)
show(codom)

TestCase{Vector{Int64}, Int64}([1], 1)TestCase{Vector{Int64}, Int64}([1, 1, 2], 1)

In [19]:
# Test case for the ninth task of determining whether a list is a palindrome.

const NinthTestCase = TestCase{Vector{Int}, Bool} # Check if a given array is a palindrome

example9 = NinthTestCase(([1,2,2,3,4,4]), false)

struct NinthTestSuiteC <: Model{Tuple{NinthTestCase, Tuple{NinthTestCase, NinthTestCase}}} end

# Test suite model for ninth task
# An object is valid, if it is a valid test case of the task, i.e. if the input list is a palindrome and expected output is true or if the input list is not a palindrome and expected output is false.
# A morphism is a tuple of domain and codomain test cases. It is valid if these conditions are satisfied: 1) domain and codomain expected outputs match. 2) Input arrays are both either palindrome or not in the domain and codomain lists. 3) If the input lists are palindromes, domain input list should appear in the middle of codomain input list. If the input lists are not palindromes, codomain input list should contain all elements of domain input list
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{NinthTestCase, Tuple{NinthTestCase, NinthTestCase}} [model::NinthTestSuiteC] begin
    Ob(x::NinthTestCase) = x.expectedOutput == (reverse(x.input) == x.input) ? x : @fail "Invalid object"

    # Morphism can add same elements to both sides of vector if the vector is a palindrome.
    # It can also add element anywhere if the vector is not a palindrome
    function Hom(f::Tuple{NinthTestCase, NinthTestCase}, dom::NinthTestCase, codom::NinthTestCase)  
        dom_input = f[1].input
        codom_input = f[2].input

        if f[1].expectedOutput != f[2].expectedOutput
            @fail "Invalid morphism"
        end

        if dom_input == reverse(dom_input) && codom_input != reverse(codom_input)
            @fail "Invalid morphism"
        end

        if dom_input != reverse(dom_input) && codom_input == reverse(codom_input)
            @fail "Invalid morphism"
        end

        if f[1].expectedOutput == true
            for i in 1:(length(codom_input) - length(dom_input) + 1) 
                if codom_input[i:(i + length(dom_input) - 1)] == dom_input
                    return f
                end
            end
        else
            if any(dom_input[i] != codom_input[i] for i in 1:length(dom_input))
                @fail "Invalid morphism"
            end
            
            return f
        end
    end

    id(x::NinthTestCase) = (x, x)

    compose(f::Tuple{NinthTestCase, NinthTestCase}, g::Tuple{NinthTestCase, NinthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{NinthTestCase, NinthTestCase}) = f[1]

    codom(f::Tuple{NinthTestCase, NinthTestCase}) = f[2]
end

##docsink#408 (generic function with 0 methods)

In [20]:
# Example test cases in the test suite of the ninth task

a = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1], true))
b = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([2,1,2], true))
c = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([3,2,1,2,3], true))

test = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1,2,1], true))

f = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((b,c),b,c)

id = GATlab.ThThinCategory.id[NinthTestSuiteC()](a)

compose = GATlab.ThThinCategory.compose[NinthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[NinthTestSuiteC()](compose)
codom = GATlab.ThThinCategory.codom[NinthTestSuiteC()](compose)

a2 = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1, 2], false))
b2 = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1, 2, 3], false))
c2 = GATlab.ThThinCategory.Ob[NinthTestSuiteC()](NinthTestCase([1, 2, 3, 4], false))

f = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((a2,b2),a2,b2)
g = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((b2,c2),b2,c2)

e = GATlab.ThThinCategory.Hom[NinthTestSuiteC()]((b2,c2),b2,c2)

GATlab.ThThinCategory.compose[NinthTestSuiteC()](f,g)


(NinthTestCase([1, 2], false), NinthTestCase([1, 2, 3, 4], false))

In [21]:
# Test case for the tenth task of removing duplicate elements from a list.

const TenthTestCase = TestCase{Vector{Int}, Vector{Int}} # Check if a given array is a palindrome

example10 = TenthTestCase(([1,2,2,3,4,4]), [1,2,3,4])

struct TenthTestSuiteC <: Model{Tuple{TenthTestCase, Tuple{TenthTestCase, TenthTestCase}}} end

# Test suite model for tenth task
# An object is valid, if it is a valid test case of the task, i.e. if input list converted to a set is equal to expected output list converted to set.
# A morphism is a tuple of domain and codomain test cases. It is valid if list of domain input is a subset of list of codomain input.  
# Composition of morphisms is valid, if codomain of the first morphism matches the domain of the second morphism.

@instance GATlab.ThThinCategory{TenthTestCase, Tuple{TenthTestCase, TenthTestCase}} [model::TenthTestSuiteC] begin
    Ob(x::TenthTestCase) = Set(x.input) == Set(x.expectedOutput) ? x : @fail "Invalid object"

    function Hom(f::Tuple{TenthTestCase, TenthTestCase}, dom::TenthTestCase, codom::TenthTestCase)
        dom_input = f[1].input
        codom_input = f[2].input

        if issubset(Set(dom_input), Set(codom_input))
            return f
        end

        @fail "Invalid morphism"
    end

    id(x::TenthTestCase) = (x, x)

    compose(f::Tuple{TenthTestCase, TenthTestCase}, g::Tuple{TenthTestCase, TenthTestCase}) = f[2] == g[1] ? (f[1], g[2]) : @fail "Invalid composition"

    dom(f::Tuple{TenthTestCase, TenthTestCase}) = f[1]

    codom(f::Tuple{TenthTestCase, TenthTestCase}) = f[2]
end

##docsink#421 (generic function with 0 methods)

In [22]:
# Example test cases in the test suite of the tenth task

a = GATlab.ThThinCategory.Ob[TenthTestSuiteC()](TenthTestCase([1], [1]))
b = GATlab.ThThinCategory.Ob[TenthTestSuiteC()](TenthTestCase([1,1], [1]))
c = GATlab.ThThinCategory.Ob[TenthTestSuiteC()](TenthTestCase([1,1,2], [1,2]))

f = GATlab.ThThinCategory.Hom[TenthTestSuiteC()]((a,b),a,b)
g = GATlab.ThThinCategory.Hom[TenthTestSuiteC()]((b,c),b,c)

id = GATlab.ThThinCategory.id[TenthTestSuiteC()](a)

composition = GATlab.ThThinCategory.compose[TenthTestSuiteC()](f,g)

dom = GATlab.ThThinCategory.dom[TenthTestSuiteC()](composition)
codom = GATlab.ThThinCategory.codom[TenthTestSuiteC()](composition)

show(dom)
show(codom)

TestCase{Vector

{Int64}, Vector{Int64}}([1], [1])TestCase{Vector{Int64}, Vector{Int64}}([1, 1, 2], [1, 2])

In [23]:
# Function that represents a random selection of an element from a vector.

function get_random(vector::Vector{T}) where T
    if isempty(vector)
        return Nothing
    end

    index = rand(1:length(vector))
    return vector[index]
end

get_random (generic function with 1 method)

In [24]:
# Model of test case sampler.
# Object is defined as a random selection of a test case from a list of possible test cases.
# Morphism is defined as a transition from one scope of possible test cases to another.

struct TestCaseSamplerC{T<:Any} <: Model{Tuple{Vector{T}, Tuple{Vector{T}, Vector{T}}}} end

@instance GATlab.ThThinCategory{Vector{T}, Tuple{Vector{T}, Vector{T}}} [model::TestCaseSamplerC{T}] where {T} begin
    Ob(vector::Vector{T}) = :(get_random($vector))

    Hom(f::Tuple{Vector{T}, Vector{T}}, dom::Vector{T}, codom::Vector{T}) = f

    id(vector::Vector{T}) = (vector, vector)

    compose(f::Tuple{Vector{T}, Vector{T}}, g::Tuple{Vector{T}, Vector{T}}) = Tuple(f[1], g[2])

    dom(f::Tuple{Vector{T}, Vector{T}}) = :(get_random($(f[1])))

    codom(f::Tuple{Vector{T}, Vector{T}}) = :(get_random($(f[2])))
end

##docsink#434 (generic function with 0 methods)

In [25]:
NumberLimit = 10

# Function for initializing arbitrarely selected test cases for the first task.

function initialize_testcases(limit::Int, condition::Function)
    testcases = [FirstTestCase((i, j, k), max(i, j, k)) for i in -limit:limit, j in -limit:limit, k in -limit:limit if condition(i, j, k)]
    return reduce(vcat,testcases)
end

testcases1 = initialize_testcases(NumberLimit, (i,j,k) -> (i != j && i != k && j != k))
testcases2 = initialize_testcases(NumberLimit, (i,j,k) -> (count(x -> x == i, (i, j, k)) == 2 || count(x -> x == j, (i, j, k)) == 2 || count(x -> x == k, (i, j, k)) == 2))
testcases3 = initialize_testcases(NumberLimit, (i,j,k) -> (i == j == k))

21-element Vector{FirstTestCase}:
 FirstTestCase((-10, -10, -10), -10)
 FirstTestCase((-9, -9, -9), -9)
 FirstTestCase((-8, -8, -8), -8)
 FirstTestCase((-7, -7, -7), -7)
 FirstTestCase((-6, -6, -6), -6)
 FirstTestCase((-5, -5, -5), -5)
 FirstTestCase((-4, -4, -4), -4)
 FirstTestCase((-3, -3, -3), -3)
 FirstTestCase((-2, -2, -2), -2)
 FirstTestCase((-1, -1, -1), -1)
 ⋮
 FirstTestCase((2, 2, 2), 2)
 FirstTestCase((3, 3, 3), 3)
 FirstTestCase((4, 4, 4), 4)
 FirstTestCase((5, 5, 5), 5)
 FirstTestCase((6, 6, 6), 6)
 FirstTestCase((7, 7, 7), 7)
 FirstTestCase((8, 8, 8), 8)
 FirstTestCase((9, 9, 9), 9)
 FirstTestCase((10, 10, 10), 10)

In [26]:
# Function defined to filter a vector based on a given criteria. Criteria is defined a function that results in a boolean.

function filter(vector::Vector{T}, criteria::Function) :: Vector{T} where {T}
    new_vector = [x for x in vector if criteria(x)]
    return length(new_vector) == 0 ? Vector{T}() : new_vector
end

filter_criteria = Function

struct TestCaseSamplerC2{T<:TestCase} <: Model{Tuple{Vector{T}, Tuple{Vector{T}, Function}}} end

# Second variant of model of test case test case sampler
# Object is defined as a random selection of a test case from a list of possible test cases.
# Morphism is defined as a filter of a vector by some criteria. The codomain of the morphism is a filtered version of the domain vector
# Composition is defined as combination of criteria of two filters.
# Identity morphism is defined as filter with criteria function that always results in true.

@instance GATlab.ThCategory{Vector{T}, Tuple{Vector{T}, Function}} [model::TestCaseSamplerC2{T}] where {T} begin
    Ob(vector::Vector{T}) = :(get_random($vector))

    Hom(f::Tuple{Vector{T}, Function}, dom::Vector{T}, codom::Vector{T}) = filter(f[1], f[2]::Function) == codom ? (f[1],f[2]) : @fail "Invalid morphism"

    id(vector::Vector{T}) = (vector, ((x) -> true)::Function)

    compose(f::Tuple{Vector{T}, Function}, g::Tuple{Vector{T}, Function}) = filter(f[1], f[2]::Function) == g[1] ? (f[1], (x) -> f[2](x) && g[2](x) ) : @fail "Invalid composition"

    dom(f::Tuple{Vector{T}, Function}) = :(get_random($(f[1])))

    codom(f::Tuple{Vector{T}, Function}) = :(get_random($(filter(f[1],f[2]::Function))))
end

##docsink#447 (generic function with 0 methods)

In [109]:
# Generation of test cases by evaluating the expression of sampler object

function generate_objects!(testsuitemodel_objects::Vector, sampler_object, count::Int, test_suite_symbol::Symbol)
    for i in 1:count
        sampled_testcase = eval(sampler_object)

        if sampled_testcase in testsuitemodel_objects
            i = i - 1
        else
            push!(testsuitemodel_objects, GATlab.ThThinCategory.Ob[eval(test_suite_symbol)()](sampled_testcase))
        end
    end
end


# Generating all morphisms in the model of test suite

function generate_morphisms(objects::Vector, test_suite_symbol::Symbol, test_case_symbol::Symbol)
    morphisms = []

    for i in 1:length(objects)
        for j in i:length(objects)
            if i != j
                try
                    new_morphism1 = GATlab.ThThinCategory.Hom[eval(test_suite_symbol)()]((objects[i],objects[j])::Tuple{eval(test_case_symbol), eval(test_case_symbol)}, objects[i], objects[j])
                    morphisms = in(new_morphism1, morphisms) ? morphisms : push!(morphisms, new_morphism1)
                catch
                end

                try
                    new_morphism2 = GATlab.ThThinCategory.Hom[eval(test_suite_symbol)()]((objects[j],objects[i])::Tuple{eval(test_case_symbol), eval(test_case_symbol)}, objects[j], objects[i])
                    morphisms = in(new_morphism2, morphisms) ? morphisms : push!(morphisms, new_morphism2)
                catch
                end
            else
                push!(morphisms, GATlab.ThThinCategory.id[eval(test_suite_symbol)()](objects[i]))
            end
        end
    end

    return morphisms
end

generate_morphisms (generic function with 3 methods)

In [84]:
# Generation of test suite for first task. Test cases are divided into few groups. Where are elements are the same, where there are two equal elements, where all elements are different. Additional group with positive integers is selected.

initial_testcases = initialize_testcases(10, (x,y,z) -> true)
all_same = filter(initial_testcases, (x) -> length(Set(x.input)) == 1)
all_same_positive = filter(all_same, (x) -> x.input[1] > 0)

two_same = filter(initial_testcases, (x) -> length(Set(x.input)) == 2)

all_different = filter(initial_testcases, (x) -> length(Set(x.input)) == 3)

terminal_testcases = filter(all_same, (x) -> false)

initial = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](initial_testcases)

filtered_objects1 = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](all_same)
filtered_objects2 = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](all_same_positive)

filtered_objects3 = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](two_same)
filtered_objects4 = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](all_different)

terminal = GATlab.ThCategory.Ob[TestCaseSamplerC2{FirstTestCase}()](terminal_testcases)

f = GATlab.ThCategory.Hom[TestCaseSamplerC2{FirstTestCase}()]((initial_testcases, (x) -> x.input[1] == x.input[2] == x.input[3]), initial_testcases, all_same)
g = GATlab.ThCategory.Hom[TestCaseSamplerC2{FirstTestCase}()]((all_same, (x) -> x.input[1] > 0), all_same, all_same_positive)

h = GATlab.ThCategory.Hom[TestCaseSamplerC2{FirstTestCase}()]((initial_testcases, (x) -> (x.input[1] == x.input[2] != x.input[3]) ||  (x.input[1] == x.input[3] != x.input[2]) || (x.input[2] == x.input[3] != x.input[1])), initial_testcases, two_same)
id = GATlab.ThCategory.id[TestCaseSamplerC2{FirstTestCase}()](initial_testcases)

e = GATlab.ThCategory.Hom[TestCaseSamplerC2{FirstTestCase}()]((initial_testcases, (x) -> (x.input[1] != x.input[2]) && ( x.input[1] != x.input[3]) && (x.input[2] != x.input[3])), initial_testcases, all_different)

comp = GATlab.ThCategory.compose[TestCaseSamplerC2{FirstTestCase}()](f,g)

codom = GATlab.ThCategory.codom[TestCaseSamplerC2{FirstTestCase}()](e)

model_objects = []

generate_objects!(model_objects, initial, 3, nameof(FirstTestSuiteC))
generate_objects!(model_objects, filtered_objects1, 3, nameof(FirstTestSuiteC))
generate_objects!(model_objects, filtered_objects2, 3, nameof(FirstTestSuiteC))
generate_objects!(model_objects, filtered_objects3, 3, nameof(FirstTestSuiteC))
generate_objects!(model_objects, filtered_objects4, 3, nameof(FirstTestSuiteC))

model_objects
morphisms = generate_morphisms(model_objects, nameof(FirstTestSuiteC), nameof(FirstTestCase))

morphisms


14-element Vector{Any}:
 (FirstTestCase((-7, -9, -7), -7), FirstTestCase((-7, -9, -7), -7))
 (FirstTestCase((0, -3, 9), 9), FirstTestCase((0, -3, 9), 9))
 (FirstTestCase((1, 0, 2), 2), FirstTestCase((1, 0, 2), 2))
 (FirstTestCase((-2, -2, -2), -2), FirstTestCase((-2, -2, -2), -2))
 (FirstTestCase((-4, -4, -4), -4), FirstTestCase((-4, -4, -4), -4))
 (FirstTestCase((2, 2, 2), 2), FirstTestCase((2, 2, 2), 2))
 (FirstTestCase((10, 10, 10), 10), FirstTestCase((10, 10, 10), 10))
 (FirstTestCase((3, 3, 3), 3), FirstTestCase((3, 3, 3), 3))
 (FirstTestCase((10, 10, 7), 10), FirstTestCase((10, 10, 7), 10))
 (FirstTestCase((-2, -2, -3), -2), FirstTestCase((-2, -2, -3), -2))
 (FirstTestCase((9, 9, 7), 9), FirstTestCase((9, 9, 7), 9))
 (FirstTestCase((3, 8, 6), 8), FirstTestCase((3, 8, 6), 8))
 (FirstTestCase((2, -8, -4), 2), FirstTestCase((2, -8, -4), 2))
 (FirstTestCase((-3, 1, 10), 10), FirstTestCase((-3, 1, 10), 10))

In [125]:
# Test suite for second task. Test cases are grouped into 2 groups: one for even numbers as inputs and other for odd numbers for inputs.

function initialize_testcases2(limit::Int)
    testcases = [SecondTestCase(i, i % 2 == 0 ? 1 : 0) for i in -limit:limit]
    return reduce(vcat,testcases)
end

# Morphisms for objects of second task because model of second task test suite has different morphisms.
function generate_morphisms2(objects::Vector)
    morphisms = []

    for i in 1:length(objects)
        for j in i:length(objects)
            if i != j
                new_morphism1 = (objects[i].input, objects[j].input - objects[i].input)
                new_morphism2 = (objects[j].input, objects[i].input - objects[j].input)

                try
                    morphisms = in(new_morphism1, morphisms) ? morphisms : push!(morphisms, GATlab.ThThinCategory.Hom[SecondTestSuiteC()](new_morphism1, objects[i], objects[j]))
                catch
                end
                try
                    morphisms = in(new_morphism2, morphisms) ? morphisms : push!(morphisms, GATlab.ThThinCategory.Hom[SecondTestSuiteC()](new_morphism2, objects[j], objects[i]))
                catch
                end
            else
                GATlab.ThThinCategory.id[SecondTestSuiteC()](objects[i])
            end
        end
    end

    return morphisms
end

initial_testcases = initialize_testcases2(50)

even = filter(initial_testcases, (x) -> x.input % 2 == 0)
odd = filter(initial_testcases, (x) -> x.input % 2 == 1)

even_ob = GATlab.ThCategory.Ob[TestCaseSamplerC2{SecondTestCase}()](even)
odd_ob = GATlab.ThCategory.Ob[TestCaseSamplerC2{SecondTestCase}()](odd)

f = GATlab.ThCategory.Hom[TestCaseSamplerC2{SecondTestCase}()]((initial_testcases, (x) -> x.input % 2 == 0), initial_testcases, even)
g = GATlab.ThCategory.Hom[TestCaseSamplerC2{SecondTestCase}()]((initial_testcases, (x) -> x.input % 2 == 1), initial_testcases, odd)

codom = GATlab.ThCategory.codom[TestCaseSamplerC2{SecondTestCase}()](g)

model_objects = []

generate_objects!(model_objects, even_ob, 10, nameof(SecondTestSuiteC))
generate_objects!(model_objects, odd_ob, 10, nameof(SecondTestSuiteC))

morphisms = generate_morphisms2(model_objects)

for i in 1:length(morphisms)
    print(GATlab.ThThinCategory.dom[SecondTestSuiteC()](morphisms[i]))
    print(morphisms[i])
    print(GATlab.ThThinCategory.codom[SecondTestSuiteC()](morphisms[i]))
    println()
end
# model_objects
# eval(even_ob)

10(10, -8)2
2(2, 8)10
10(10, 2)12
12(12, -2)10
10(10, -22)-12
-12(-12, 22)10
10(10, 14)24
24(24, -14)10
10(10, -48)-38
-38(-38, 48)10
10(10, -44)-34
-34(-34, 44)10
10(10, -4)6
6(6, 4)10
10(10, 24)34
34(34, -24)10
2(2, 10)12
12(12, -10)2
2(2, -14)-12
-12(-12, 14)2
2(2, 22)24
24(24, -22)2
2(2, -40)-38
-38(-38, 40)2
2(2, -36)-34
-34(-34, 36)2
2(2, 4)6
6(6, -4)2
2(2, 32)34
34(34, -32)2
12(12, -24)-12
-12(-12, 24)12
12(12, 12)24
24(24, -12)12
12(12, -50)-38
-38(-38, 50)12
12(12, -46)-34
-34(-34, 46)12
12(12, -6)6
6(6, 6)12
12(12, 22)34
34(34, -22)12
-12(-12, 36)24
24(24, -36)-12
-12(-12, -26)-38
-38(-38, 26)-12
-12(-12, -22)-34
-34(-34, 22)-12
-12(-12, 18)6
6(6, -18)-12
-12(-12, 46)34
34(34, -46)-12
24(24, -62)-38
-38(-38, 62)24
24(24, -58)-34
-34(-34, 58)24
24(24, -18)6
6(6, 18)24
24(24, 10)34
34(34, -10)24
-38(-38, 4)-34
-34(-34, -4)-38
-38(-38, 44)6
6(6, -44)-38
-38(-38, 72)34
34(34, -72)-38
-34(-34, 40)6
6(6, -40)-34
-34(-34, 68)34
34(34, -68)-34
6(6, 28)34
34(34, -28)6
25(25, 20)45
45(